# Day 5: Your First Agent - Building Custom AI Agents

## What You'll Learn Today

Welcome to Day 5! Today is a **major milestone** - you'll learn how to build your own AI agents from scratch!

**What's an Agent?**
Think of an agent as an AI assistant with a specific role and workflow. Just like:
- A **translator agent** that only translates languages
- A **summarizer agent** that condenses long documents
- A **customer support agent** that answers specific product questions

### Today's Learning Path:
1. **Understand the Agent hierarchy** - How agents inherit capabilities
2. **Explore the base Agent class** - The foundation of all agents
3. **Create simple custom agents** - EchoAgent, TranslatorAgent
4. **Learn the _run() method** - The heart of every agent
5. **Understand FnCallAgent** - Agents that can use tools
6. **Build tool-using agents** - Combining LLM + tools
7. **See practical patterns** - Real-world agent examples

Let's build your first agent! 🤖

---
## Part 1: Configure Our Environment

Same Fireworks API setup as previous days.

In [ ]:
# ================================================
# FIREWORKS API CONFIGURATION
# ================================================
import os
import json

# Set API credentials
os.environ['FIREWORKS_API_KEY'] = 'fw_3ZSpUnVR78vs38jJtyewjcWk'

# Standard configuration for Fireworks Qwen3-235B-A22B-Thinking
llm_cfg_fireworks = {
    'model': 'accounts/fireworks/models/qwen3-235b-a22b-thinking-2507',
    'model_server': 'https://api.fireworks.ai/inference/v1',
    'api_key': os.environ['FIREWORKS_API_KEY'],
    'generate_cfg': {
        'max_tokens': 32768,
        'temperature': 0.6,
    }
}

# Use this as default llm_cfg
llm_cfg = llm_cfg_fireworks

print('✅ Configured for Fireworks API')
print(f'   Model: Qwen3-235B-A22B-Thinking-2507')
print(f'   Max tokens: 32,768')

---
## Part 2: Understanding the Agent Hierarchy

### The Agent Family Tree

Qwen-Agent has a clear inheritance hierarchy:

```
Agent (base class)
  ↓
  ├─ FnCallAgent (adds tool-calling ability)
  │   ↓
  │   ├─ Assistant (general-purpose agent with RAG)
  │   ├─ ReActChat (uses ReAct prompting)
  │   └─ ParallelDocQA (specialized for document Q&A)
  │
  └─ Your Custom Agent (you can extend any of these!)
```

### What Each Level Provides

| Class | What It Adds | When to Use |
|-------|-------------|-------------|
| **Agent** | Basic structure, LLM integration | Simple agents without tools |
| **FnCallAgent** | Tool calling, function execution | Agents that need to DO things |
| **Assistant** | RAG, file handling, memory | General-purpose assistants |

**Think of it like building blocks:**
- Start with **Agent** for the foundation
- Add **FnCallAgent** when you need tools
- Use **Assistant** when you need the full suite

Let's explore each level!

---
## Part 3: The Base Agent Class

### What is the Agent Class?

The `Agent` class is an **abstract base class** (ABC). This means:
- You can't use it directly (you must subclass it)
- It defines the interface all agents must follow
- It provides common functionality (LLM integration, tool management)

### Key Components of Agent

```python
class Agent(ABC):
    def __init__(self, llm, function_list, system_message, name, description):
        self.llm = ...              # The LLM model
        self.function_map = ...     # Available tools
        self.system_message = ...   # System prompt
        self.name = ...             # Agent name
        self.description = ...      # What this agent does
    
    @abstractmethod
    def _run(self, messages, **kwargs):
        """This MUST be implemented by subclasses"""
        pass
    
    def run(self, messages, **kwargs):
        """Public interface - handles type conversions and calls _run()"""
        pass
```

**The key insight:** 
- `run()` is the public interface (you call this)
- `_run()` is the private implementation (you override this)

Let's see this in practice!

In [ ]:
# Let's examine the Agent base class
from qwen_agent import Agent
import inspect

print("Agent class signature:")
print(inspect.signature(Agent.__init__))

print("\nAgent methods:")
for name, method in inspect.getmembers(Agent, predicate=inspect.isfunction):
    if not name.startswith('_') or name in ['_run', '_call_llm', '_call_tool']:
        print(f"  - {name}")

---
## Part 4: Your First Custom Agent - EchoAgent

### The Simplest Possible Agent

Let's create an agent that simply echoes back what you say (without using an LLM).

This will teach us:
1. How to inherit from Agent
2. How to implement `_run()`
3. How to yield responses
4. The Message format

In [ ]:
from qwen_agent import Agent
from qwen_agent.llm.schema import Message
from typing import List, Iterator

class EchoAgent(Agent):
    """A simple agent that echoes back what you say"""
    
    def _run(self, messages: List[Message], **kwargs) -> Iterator[List[Message]]:
        """
        The _run method MUST:
        1. Accept messages (conversation history)
        2. Yield responses (as List[Message])
        3. Return an iterator (for streaming)
        """
        # Get the last user message
        last_message = messages[-1]
        user_content = last_message.content
        
        # Create a response
        response = Message(
            role='assistant',
            content=f"You said: {user_content}"
        )
        
        # Yield the response (must be a list!)
        yield [response]

# Create an instance (no LLM needed!)
echo_bot = EchoAgent(
    name='EchoBot',
    description='An agent that echoes your messages'
)

# Test it
messages = [{'role': 'user', 'content': 'Hello, agent!'}]
print("Testing EchoAgent:\n")
for response in echo_bot.run(messages):
    print(f"Response: {response[0]['content']}")

### Understanding the _run() Method

**Why yield instead of return?**

Yielding allows for **streaming responses**:
- The UI can show partial results as they arrive
- Better user experience ("typing" effect)
- Works with both simple and complex agents

**Why yield a list?**

Because an agent might:
- Generate multiple messages (e.g., assistant + function call)
- Stream partial updates
- Return conversation history

**Pattern:**
```python
def _run(self, messages, **kwargs):
    # 1. Process input
    # 2. Generate response
    # 3. Yield as List[Message]
    yield [response]
```

---
## Part 5: Adding LLM Intelligence - TranslatorAgent

### Creating an Agent that Uses the LLM

Now let's create a more useful agent that actually uses the LLM to translate text!

We'll learn:
1. How to use `self.llm.chat()`
2. How to add a system message
3. How to stream LLM responses

In [ ]:
class TranslatorAgent(Agent):
    """An agent that translates text to a target language"""
    
    def __init__(self, target_language='French', **kwargs):
        # Call parent __init__
        super().__init__(
            name=f'Translator-{target_language}',
            description=f'Translates text to {target_language}',
            system_message=f'You are a professional translator. Translate all user input to {target_language}. Only output the translation, nothing else.',
            **kwargs
        )
        self.target_language = target_language
    
    def _run(self, messages: List[Message], **kwargs) -> Iterator[List[Message]]:
        # The messages parameter already contains the system message (added by parent's run() method)
        # So we can directly call the LLM without adding it again
        
        # Call the LLM - using stream=True to properly yield responses
        # Each iteration yields a List[Message] as required
        for response in self.llm.chat(messages=messages, stream=True):
            yield response

# Create a translator to French
french_translator = TranslatorAgent(
    target_language='French',
    llm=llm_cfg
)

# Test it
print("Testing TranslatorAgent:\n")
messages = [{'role': 'user', 'content': 'Hello, how are you today?'}]

# Collect responses and show only the final one
final_response = None
for response in french_translator.run(messages):
    if response:
        final_response = response[-1]['content']

print(f"Translation: {final_response}")

### Try Different Languages!

In [ ]:
# Create translators for different languages
spanish_translator = TranslatorAgent(target_language='Spanish', llm=llm_cfg)
japanese_translator = TranslatorAgent(target_language='Japanese', llm=llm_cfg)

test_text = "The weather is beautiful today!"
print(f"Original: {test_text}\n")

# Spanish - collect final response only
spanish_result = None
for response in spanish_translator.run([{'role': 'user', 'content': test_text}]):
    if response:
        spanish_result = response[-1]['content']
print(f"Spanish: {spanish_result}")

# Japanese - collect final response only 
japanese_result = None
for response in japanese_translator.run([{'role': 'user', 'content': test_text}]):
    if response:
        japanese_result = response[-1]['content']
print(f"Japanese: {japanese_result}")

---
## Part 6: Understanding FnCallAgent

### What is FnCallAgent?

**FnCallAgent** is the base class for agents that can **call tools/functions**.

It adds:
- ✅ Automatic tool management (`function_map`)
- ✅ Function calling loop (call LLM → detect function → execute → repeat)
- ✅ Helper methods (`_call_llm()`, `_call_tool()`)
- ✅ Memory management for file handling

### The FnCallAgent Loop

```
User Input
   ↓
Call LLM (with function list)
   ↓
LLM decides: Use function or answer directly?
   ↓                           ↓
Function call            Direct answer
   ↓                           ↓
Execute function           Return to user
   ↓
Add result to conversation
   ↓
Call LLM again (repeat up to MAX_LLM_CALL_PER_RUN times)
```

**This is automatic! FnCallAgent implements this loop for you.**

In [ ]:
# Let's examine FnCallAgent
from qwen_agent.agents import FnCallAgent

print("FnCallAgent signature:")
print(inspect.signature(FnCallAgent.__init__))

print("\nKey differences from Agent:")
print("  - Has function_list parameter")
print("  - Manages function_map automatically")
print("  - Implements tool-calling loop in _run()")
print("  - Provides _call_llm() and _call_tool() helpers")

### Using FnCallAgent Directly

You can actually use FnCallAgent directly (it's not abstract like Agent)!

In [ ]:
# Create a FnCallAgent with code_interpreter
calculator_agent = FnCallAgent(
    llm=llm_cfg,
    function_list=['code_interpreter'],
    name='Calculator',
    description='An agent that can do calculations',
    system_message='You are a helpful calculator. Use code_interpreter to do exact calculations.'
)

# Test it
messages = [{'role': 'user', 'content': 'What is 12345 * 67890?'}]
print("User: What is 12345 * 67890?\n")

for response in calculator_agent.run(messages):
    for msg in response:
        if msg.get('function_call'):
            print(f"[Calling tool: {msg['function_call']['name']}]")
        elif msg.get('role') == 'assistant' and msg.get('content'):
            print(f"Assistant: {msg['content']}\n")

---
## Part 7: Creating a Custom FnCallAgent

### When to Extend FnCallAgent

You extend FnCallAgent when you want to:
- Customize the tool-calling behavior
- Add pre/post-processing
- Implement a specific workflow
- Add custom logic around function calls

### Example: SummarizerAgent

Let's create an agent that:
1. Uses the LLM to summarize text
2. Always formats output as bullet points
3. Can optionally use code_interpreter for analysis

In [ ]:
from qwen_agent.agents import FnCallAgent
from qwen_agent.llm.schema import Message
from typing import Iterator, List, Literal
import copy

class SummarizerAgent(FnCallAgent):
    """An agent specialized in summarizing text"""
    
    def __init__(self, **kwargs):
        # Can use code_interpreter if needed
        super().__init__(
            function_list=['code_interpreter'],
            system_message='You are a professional summarizer. Provide concise bullet-point summaries. If analyzing data, use code_interpreter.',
            name='Summarizer',
            description='Summarizes text and data into bullet points',
            **kwargs
        )
    
    def _run(self, messages: List[Message], lang: Literal['en', 'zh'] = 'en', **kwargs) -> Iterator[List[Message]]:
        # Add custom instruction to always format as bullets
        messages = copy.deepcopy(messages)
        last_msg = messages[-1]
        last_msg.content = f"{last_msg.content}\n\nProvide the summary as bullet points."
        
        # Call the parent's _run (which does the tool-calling loop)
        yield from super()._run(messages, lang=lang, **kwargs)

# Create the agent
summarizer = SummarizerAgent(llm=llm_cfg)

print("SummarizerAgent created!")
print(f"Available tools: {list(summarizer.function_map.keys())}")

### Test the SummarizerAgent

In [ ]:
# Test with a paragraph
long_text = """
Artificial Intelligence has revolutionized many industries in recent years. Machine learning 
algorithms can now recognize patterns in data with superhuman accuracy. Deep learning, a subset 
of machine learning, uses neural networks with multiple layers to process information. These 
technologies have applications in healthcare, finance, autonomous vehicles, and natural language 
processing. However, AI also raises important ethical questions about privacy, bias, and the 
future of work. As AI continues to advance, society must carefully consider both its benefits 
and potential risks.
"""

messages = [{
    'role': 'user',
    'content': f'Summarize this text: {long_text}'
}]

print("Asking SummarizerAgent to summarize...\n")
for response in summarizer.run(messages):
    if response:
        last_msg = response[-1]
        if last_msg.get('role') == 'assistant' and last_msg.get('content') and not last_msg.get('function_call'):
            print("Summary:")
            print(last_msg['content'])

---
## Part 8: Helper Methods - _call_llm() and _call_tool()

### Understanding the Helpers

FnCallAgent provides two important helper methods:

#### 1. _call_llm(messages, functions, extra_generate_cfg)
- Calls the LLM with function definitions
- Handles streaming
- Returns an iterator of responses

#### 2. _call_tool(tool_name, tool_args, **kwargs)
- Executes a tool by name
- Handles parameter parsing
- Returns tool result as string

These are **protected methods** (start with `_`) because they're meant for internal use by agents.

### Example: Manual Tool Calling

Let's see how to use these helpers directly (advanced use case).

In [ ]:
class StepByStepAgent(FnCallAgent):
    """An agent that shows each step explicitly"""
    
    def _run(self, messages: List[Message], **kwargs) -> Iterator[List[Message]]:
        print("Step 1: Calling LLM...")
        
        # Manually call LLM
        functions = [func.function for func in self.function_map.values()]
        llm_response = []
        for llm_response in self._call_llm(messages=messages, functions=functions):
            yield llm_response
        
        if not llm_response:
            return
        
        # Check if LLM wants to call a function
        last_msg = llm_response[-1]
        if last_msg.get('function_call'):
            print("Step 2: LLM requested function call")
            
            fn_name = last_msg['function_call']['name']
            fn_args = last_msg['function_call']['arguments']
            
            print(f"Step 3: Executing {fn_name}...")
            
            # Manually call tool
            result = self._call_tool(fn_name, fn_args, messages=messages)
            
            print(f"Step 4: Got result (first 100 chars): {result[:100]}...")
            
            # Add function result to messages
            from qwen_agent.llm.schema import FUNCTION
            fn_msg = Message(
                role=FUNCTION,
                name=fn_name,
                content=result
            )
            messages.extend(llm_response + [fn_msg])
            
            print("Step 5: Calling LLM again with function result...")
            
            # Call LLM again with the result
            final_response = []
            for final_response in self._call_llm(messages=messages, functions=functions):
                yield llm_response + [fn_msg] + final_response
        else:
            print("Step 2: LLM provided direct answer (no function call)")

# Test it
step_agent = StepByStepAgent(
    llm=llm_cfg,
    function_list=['code_interpreter'],
    name='StepByStep'
)

print("Testing StepByStepAgent:\n")
messages = [{'role': 'user', 'content': 'Calculate the square root of 144'}]

for response in step_agent.run(messages):
    pass  # Just let it print the steps

if response:
    print(f"\nFinal answer: {response[-1].get('content', '')}")

---
## Part 9: Practical Agent Patterns

### Pattern 1: The Specialist Agent

An agent with a narrow, well-defined purpose.

In [ ]:
class PythonTutorAgent(FnCallAgent):
    """An agent that teaches Python by running code examples"""
    
    def __init__(self, **kwargs):
        super().__init__(
            function_list=['code_interpreter'],
            system_message=(
                "You are a Python programming tutor. When explaining concepts, "
                "always provide working code examples using code_interpreter. "
                "Run the code to show the output, then explain what happened."
            ),
            name='PythonTutor',
            description='Teaches Python with executable examples',
            **kwargs
        )

# Create and test
tutor = PythonTutorAgent(llm=llm_cfg)
messages = [{'role': 'user', 'content': 'Explain list comprehensions with an example'}]

print("User: Explain list comprehensions with an example\n")
for response in tutor.run(messages):
    if response:
        last = response[-1]
        if last.get('role') == 'assistant' and last.get('content') and not last.get('function_call'):
            print(f"Tutor: {last['content']}")

### Pattern 2: The Validator Agent

An agent that validates and corrects input.

In [ ]:
class CodeValidatorAgent(FnCallAgent):
    """An agent that validates Python code"""
    
    def __init__(self, **kwargs):
        super().__init__(
            function_list=['code_interpreter'],
            system_message=(
                "You are a code validator. When given Python code:\n"
                "1. Run it with code_interpreter\n"
                "2. Report if it works or has errors\n"
                "3. If errors exist, suggest fixes\n"
                "Keep responses concise."
            ),
            name='CodeValidator',
            **kwargs
        )

validator = CodeValidatorAgent(llm=llm_cfg)

# Test with buggy code
buggy_code = """
def calculate_average(numbers):
    total = sum(numbers)
    return total / len(numbers)

result = calculate_average([])  # Bug: empty list!
print(result)
"""

messages = [{'role': 'user', 'content': f'Validate this code:\n```python\n{buggy_code}\n```'}]

print("Testing CodeValidatorAgent with buggy code:\n")
for response in validator.run(messages):
    if response:
        last = response[-1]
        if last.get('role') == 'assistant' and last.get('content') and not last.get('function_call'):
            print(f"Validator: {last['content']}")

---
## Part 10: Comparing Agent Classes

### Quick Reference Guide

Let's compare all the agent types we've learned:

In [ ]:
comparison_table = """
┌─────────────────┬──────────────┬──────────────┬────────────────────────┐
│ Agent Type      │ Can Use LLM? │ Can Use Tools│ When to Use            │
├─────────────────┼──────────────┼──────────────┼────────────────────────┤
│ Agent (base)    │ Yes          │ No           │ Simple LLM-only agents │
│ FnCallAgent     │ Yes          │ Yes          │ Agents that need tools │
│ Assistant       │ Yes          │ Yes + RAG    │ General assistants     │
│ ReActChat       │ Yes          │ Yes (ReAct)  │ Step-by-step reasoning │
│ Custom Agent    │ Your choice  │ Your choice  │ Specialized workflows  │
└─────────────────┴──────────────┴──────────────┴────────────────────────┘
"""
print(comparison_table)

print("\nDecision Tree:")
print("""  
Need tools? ─── No ──→ Extend Agent
      │
     Yes
      │
      ↓
Need RAG/Files? ─── No ──→ Use/Extend FnCallAgent
      │
     Yes
      │
      ↓
Use Assistant (covered in Day 8!)
""")

---
## Part 11: Practice Exercises

Now it's your turn! Try these exercises.

### Exercise 1: Create a ReverseAgent

Create an agent that reverses the user's input (without using LLM).

Example:
- Input: "hello world"
- Output: "dlrow olleh"

In [ ]:
# TODO: Implement ReverseAgent
# Hint: Extend Agent and implement _run()
# Hint: Use string slicing [::-1]

# Your code here:
# class ReverseAgent(Agent):
#     ...

### Exercise 2: Create a PoetAgent

Create an agent that:
- Takes any input
- Converts it into a haiku (5-7-5 syllable poem)
- Uses the LLM

In [ ]:
# TODO: Implement PoetAgent
# Hint: Extend Agent, use self.llm.chat()
# Hint: Set a good system_message about haiku format

# Your code here:
# ...

### Exercise 3: Create a MathTutorAgent

Create an agent that:
- Solves math problems using code_interpreter
- Shows the step-by-step working
- Explains the concept

Extend FnCallAgent!

In [ ]:
# TODO: Implement MathTutorAgent
# Hint: Extend FnCallAgent
# Hint: Use function_list=['code_interpreter']
# Hint: Write a good system_message

# Your code here:
# ...

### Exercise 4: Create a DebuggerAgent

Create an agent that:
1. Takes Python code as input
2. Runs it with code_interpreter
3. If it has errors, explains what went wrong
4. Suggests a fix
5. Tests the fix

In [ ]:
# TODO: Implement DebuggerAgent
# This is more advanced - you might need to override _run()
# to implement the multi-step workflow

# Your code here:
# ...

---
## Summary: What You Learned Today

### Core Concepts

✅ **Agent hierarchy**: Agent → FnCallAgent → Assistant (and custom agents)

✅ **Base Agent class**: Abstract base with run() and _run() methods

✅ **Creating custom agents**: Extend Agent or FnCallAgent and implement _run()

✅ **The _run() method**: Core implementation that yields List[Message]

✅ **FnCallAgent**: Automatic tool-calling loop for function-using agents

✅ **Helper methods**: _call_llm() and _call_tool() for manual control

✅ **Agent patterns**: Specialist, Validator, Multi-tool agents

### Key Takeaways

1. **All agents follow the same pattern** - implement _run(), yield responses
2. **Choose the right base class** - Agent for simple, FnCallAgent for tools
3. **System messages matter** - they define your agent's personality and behavior
4. **Yielding enables streaming** - better UX than returning complete responses
5. **You can mix and match** - LLM + tools + custom logic

### What's Next?

**Tomorrow (Day 6)**: We'll dive deep into **Function Calling**!

You'll learn:
- Direct LLM function calling (without agents)
- fncall_prompt_type ('qwen' vs 'nous')
- function_choice parameter
- Parallel function calls
- Error handling in function calls

### Additional Resources

- 📖 Agent Source Code: `/qwen_agent/agent.py`
- 🔧 FnCallAgent Source: `/qwen_agent/agents/fncall_agent.py`
- 💡 Examples: `/examples/function_calling.py`

---

**Congratulations! 🎉 You can now create custom AI agents from scratch!**